In [21]:
from helium10Scrapper.path import PROJECT_ROOT

In [22]:
import requests
import json
import pandas as pd
import time

In [23]:
ACCOUNT_ID = 1545862407
BEARER_TOKEN = "Bearer MoweC58CefslSCzS2YdPWOV6etiWPracUq-cYbaNYBYN08XkXK5kBy2C10tMnhKK347ea2396c6bec33cb4266a63079bf0f"

In [24]:
def get_cerebro_query_result(cerebro_query_id):
    cerebro_get_query_endpoint = f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single/{cerebro_query_id}/data"
    params = {
        "accountId":ACCOUNT_ID,
        "include-all":0,
        "include-any":1,
        "page":1,
        "per_page":1000,
        "sort":"-iq"
    }

    headers = {"Authorization":f"Bearer {BEARER_TOKEN}"}
    r = requests.get(cerebro_get_query_endpoint, params=params, headers=headers)
    return r

In [25]:
r = get_cerebro_query_result(121784347)

In [26]:
cerebro_result_dict = {121784347:r}

In [27]:
serialized_r = r.json()

In [28]:
serialized_r["data"]["tableData"][0]

{'phraseId': 98980933,
 'sponsoredPosition': None,
 'sponsoredScrapedAt': None,
 'matchType': ['ppc'],
 'organicPosition': None,
 'organicScrapedAt': None,
 'phrase': 'lc2i',
 'resultsNumber': 53,
 'resultsNumberOver': False,
 'resultsNumberUpdatedAt': 1698387335,
 'impressionExact30': 1856,
 'exactTitleMatchProductsCount': 11,
 'iq': 35018.8679245283,
 'sponsoredAsins': '220',
 'amazonChoice': 'B0BRT9L66W',
 'currentAsin': 'B07H9M94B5',
 'amzSuggestedPosition': 38,
 'amzSuggestedScrapedAt': 1698481763,
 'searchVolumeTrend30': -6,
 'newCprExact': 29.9725,
 'newCprBroad': 29.9725,
 'monthlySales': 114,
 'clickShareRate': None,
 'conversionShareRate': None,
 'cpc': 64,
 'highCpc': 70,
 'lowCpc': 56,
 'searchFrequencyRank': None,
 'octoberSeasonalSearchVolume': None,
 'decemberSeasonalSearchVolume': None,
 'novemberSeasonalSearchVolume': None}

In [35]:
def format_cerebro_result_dict(dict):
    data = []
    index = []
    for asin, response in dict.items():
        serialized_response= response.json()
        try:
            search_id = serialized_response["data"]["searchId"]
            for iq_score_rank, record in enumerate(
                serialized_response["data"]["tableData"][:20]
            ):
                index.append((search_id, asin, iq_score_rank))
                data.append(record)
        except KeyError as e:
            print(str(e))
            continue

    df = pd.DataFrame(
        data,
        index=pd.MultiIndex.from_tuples(index, names=['search_id', 'asin', 'iq_score_rank'])
    )[["phrase", "iq", "impressionExact30", "searchVolumeTrend30"]]
    df.columns = ["phrase", "iq_score", "search_volume", "search_volume_trend_30"]

    return df

In [36]:
df = format_cerebro_result_dict(cerebro_result_dict)

In [37]:
df

phrase  \
search_id asin      iq_score_rank                                     
121784347 121784347 0                                          lc2i   
                    1                      epicenter mexico edition   
                    2                epicentro para sonido de carro   
                    3                             clarion equalizer   
                    4                        epicenter audiocontrol   
                    5                                     epicenter   
                    6              ecualizador de sonido para carro   
                    7                            bocinas para carro   
                    8                                   epic center   
                    9                             clarion crossover   
                    10                                         ds18   
                    11                         restaurador de bajos   
                    12                                 utv soundbar   
                    13                                    psg audio   
                    14                        epicentro soundstream   
                    15                          equalizer car audio   
                    16                                   sony xplod   
                    17                       epicentro audiocontrol   
                    18                               devialet dione   
                    19                                utv sound bar   

                                       iq_score  search_volume  \
search_id asin      iq_score_rank                                
121784347 121784347 0              35018.867925           1856   
                    1              32800.000000            164   
                    2              23304.347826           1608   
                    3              19303.278689           2355   
                    4              18700.892857           4189   
                    5              18130.193906           6545   
                    6              13300.000000           1862   
                    7              10289.617486           3766   
                    8               9333.333333            280   
                    9               8968.750000            861   
                    10              6549.000000           6549   
                    11              6171.875000            395   
                    12              6124.277457           2119   
                    13              6031.746032            380   
                    14              4961.538462            387   
                    15              4902.735562           1613   
                    16              4567.010309            443   
                    17              4563.106796            470   
                    18              4529.411765             77   
                    19              4342.618384           1559   

                                   search_volume_trend_30  
search_id asin      iq_score_rank                          
121784347 121784347 0                                  -6  
                    1                                   3  
                    2                                  13  
                    3                                  25  
                    4                                 111  
                    5                                  20  
                    6                                  25  
                    7                                  -5  
                    8                                 -11  
                    9                                 -22  
                    10                                 11  
                    11                                  3  
                    12                                266  
                    13                                 42  
                    14                                  1  
                    15   

In [29]:
take_a_good_look = df.copy(deep=True)

In [39]:
take_a_good_look.drop(columns=[
    "sponsoredPosition", "sponsoredScrapedAt", "matchType", "resultsNumberUpdatedAt", "sponsoredAsins",
    "cpc", "highCpc", "lowCpc", "octoberSeasonalSearchVolume", "novemberSeasonalSearchVolume", "decemberSeasonalSearchVolume",
])

phraseId organicPosition organicScrapedAt  \
search_id asin      iq_score_rank                                               
121784347 121784347 0               98980933            None             None   
                    1              213147340            None             None   
                    2              212925770            None             None   
                    3                5523213            None             None   
                    4               36195507            None             None   
                    5               32254166            None             None   
                    6                4726315            None             None   
                    7               85693233            None             None   
                    8              105821662            None             None   
                    9               10544069            None             None   
                    10              79260863            None             None   
                    11              51412286            None             None   
                    12              84009191            None             None   
                    13             213331878            None             None   
                    14              87636947            None             None   
                    15               5523219            None             None   
                    16              28573573            None             None   
                    17              87636940            None             None   
                    18             212240591            None             None   
                    19              32984128            None             None   

                                                             phrase  \
search_id asin      iq_score_rank                                     
121784347 121784347 0                                          lc2i   
                    1                      epicenter mexico edition   
                    2                epicentro para sonido de carro   
                    3                             clarion equalizer   
                    4                        epicenter audiocontrol   
                    5                                     epicenter   
                    6              ecualizador de sonido para carro   
                    7                            bocinas para carro   
                    8                                   epic center   
                    9                             clarion crossover   
                    10                                         ds18   
                    11                         restaurador de bajos   
                    12                                 utv soundbar   
                    13                                    psg audio   
                    14                        epicentro soundstream   
                    15                          equalizer car audio   
                    16                                   sony xplod   
                    17                       epicentro audiocontrol   
                    18                               devialet dione   
                    19                                utv sound bar   

                                   resultsNumber  resultsNumberOver  \
search_id asin      iq_score_rank                                     
121784347 121784347 0                         53              False   
                    1                          5              False   
                    2                         69              False   
                    3                        122              False   
                    4                        224              False   
                    5                        361              False   
                    6                        140              False   
                    7                        366   